## Exercise 3

### Descrição do Problema

O objetivo deste exercício é desenvolver uma AEAD com "Tweakable Block Ciphers”", 
sendo a cifra usada do tipo AES-256 ou o ChaCha20.

Esta cifra será usada num canal privade de informação assíncrona com acordo de chaves.
A autenticação dos agentes será com:
 - “X448 key exchange”
 - “Ed448 Signing&Verification” 
 
... e deve haver uma fase de confirmação da chave acordada.


Ou seja,

Use o “package” Cryptography para
1. Implementar uma AEAD com “Tweakable Block Ciphers” conforme está descrito na última secção do texto +Capítulo 1: Primitivas Criptográficas Básicas.  A cifra por blocos primitiva, usada para gerar a “tweakable block cipher”, é o AES-256 ou o ChaCha20.
2. Use esta cifra para construir um canal privado de informação assíncrona com acordo de chaves feito com “X448 key exchange” e “Ed448 Signing&Verification” para autenticação  dos agentes. Deve incluir uma fase de confirmação da chave acordada.

In [1]:
#Imports necessários
import os
import asyncio
import random
from pickle import dumps, loads
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import x448, ed448
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.asymmetric.x448 import X448PrivateKey
from cryptography.hazmat.primitives.asymmetric.ed448 import Ed448PrivateKey
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

# Para a porção de código associada com as queues (na comunicação entre participantes)
import nest_asyncio

nest_asyncio.apply()


import warnings
warnings.simplefilter("default") 

## Criação das chaves privadas/públicas

O acordo de chaves, como indicado no enunciado, deve ser feito com __"X448 key exchange"__.

Para tal é necessário que cada um dos agentes criem as suas próprias chaves (privada-pública).

Neste caso será usando a __curva448__, como 
apresentado na API:
- https://cryptography.io/en/latest/hazmat/primitives/asymmetric/x448/

A chave resultante será usada no decorrer da comunicação assíncrona entre agentes para __cifrar/decifrar__ os dados.

In [2]:
def generateKeys():
    # Generate private key for exchange
    prv_key = x448.X448PrivateKey.generate()
    
    # Generate public key thorugh private key
    pub_key = prv_key.public_key()
    
    return prv_key, pub_key
    

## Criação das chaves partilhadas

Após ter as chaves de cada agente (privada/pública), podemos estabelecer
o _exchange_, i.e o acordo entre ambos agentes,
sobre um segredo partilhado.

A _shared_key_ criada, tal como recomendado na API, deve ser passada por uma função de derivação, no âmbito de a tornar mais segura; adicionando mais informações à chave para destruir qualquer estrutura que possa ser criada.

Será criada uma _shared_key_, a ser usada no âmbito da cifra. 

In [3]:
def generateShared(prv_key, peer_key):
    
    peer_cipher_key = x448.X448PublicKey.from_public_bytes(peer_key)
    
    # Gerar uma chave partilha para cifra
    cipher_key = prv_key.exchange(peer_cipher_key)
    
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=16, #32,
        salt=None,
        info=b'handshake data',
    ).derive(cipher_key)
    
    return derived_key

## Assinar mensagem

A autenticação dos agentes é realizada com 
__“Ed448 Signing&Verification”__; sendo este um 
algoritmo de assinaturas com recurso ao __EdDSA__ (_Edwards-curve Digital Signature Algorithm_).

Para tal é necessário um novo par de chaves (privada/pública), do tipo _Ed448_, para cada um dos agentes.

Para recorrer aos meios apresentados em:
- https://cryptography.io/en/latest/hazmat/primitives/asymmetric/ed448/

As chaves públicas devem ser partilhadas por ambos, de 
modo a validar a __assinatura__ da mensagem que receberam.
Sendo a assinatura realizada através da chave privada do agente,
a enviar a mensagem.

Criou-se um método particular para gerar as chaves de autenticação dos agentes, para facilitar e tornar o código mais _readable_.

In [4]:
def generateSignKeys():
    
    ## Chave privada para assinar
    private_key = Ed448PrivateKey.generate()
    
    ## Chave pública para autenticar
    public_key = private_key.public_key()
   
    return private_key, public_key
    
def signMsg(prv_key, msg):
    
    signature = prv_key.sign(msg)
    
    return signature

## Cifrar mensagem

In [5]:
def pad (block_plaintext, length):
    last_block = block_plaintext[-1] # Get the last block of the list
    len_last_block = len(last_block)
    
    for _ in range (len_last_block, length): # Adds the value 0 until the size the last block is 0
        last_block += b"\x00"
    
    block_plaintext[-1] = last_block
    
    return block_plaintext, len_last_block

    
def tpbc (tweak, key, block, iv):
    tweaked_key = tweak + key
    cipher = Cipher(algorithms.AES256(tweaked_key), modes.CBC(iv))
    encryptor = cipher.encryptor()
    ct_block = encryptor.update(block) + encryptor.finalize()
    return ct_block

# nounce: bytes, ctr: bytes, block_plaintext: bytes, key: bytes, auth: bytes
def tweakable_first_blocks(nounce, ctr, block_plaintext, key, auth, iv):
    ct = b""
    zero = b"\x00"
    
    for elem in (block_plaintext[:-1]):        
        tweak = nounce + ctr + zero
        
        c_i = tpbc(tweak, key, elem, iv)
        ct += c_i
        
        len_ctr = len(ctr)
        ctr_int = int.from_bytes(ctr, 'big')
        ctr_int += 1
        ctr = ctr_int.to_bytes(len_ctr, 'big')
        
        aux = b""
        for x,y in zip(auth, elem):
            word = x ^ y
            aux += word.to_bytes(1, 'big')
        
        auth = aux
        
    return ctr, auth, ct
    
def encrypt(msg, key):
    NBytes = 16; size_nounce = 8
    
    digest = hashes.Hash(hashes.SHA3_256())
    nounce_temp = digest.finalize()
    nounce = nounce_temp[:size_nounce] # array of bytes with size 8
    
    ctr_i = os.urandom(size_nounce-1) # array of bytes with size 7
    ctr = ctr_i 
    
    iv = os.urandom(16)
        
    block_plaintext = [bytes(msg[i:i+NBytes], 'utf8') for i in range(0, len(msg), NBytes)]  # List of block of bytes. Block size 16
    
    block_plaintext, len_last_block = pad(block_plaintext, NBytes) # blocks de bytes de tam 16, tamanho original do ultimo bloco 
    
    # i = 0 ... m - 1    
    auth = b"" 
    for _ in range (NBytes): # array of bytes with size 16 bytes
        auth += b"\x00"

    ctr, auth, ct = tweakable_first_blocks(nounce, ctr, block_plaintext, key, auth, iv) # ctr: bytes; auth: str ; ct: bytes

    # i = m
    tweak = nounce + ctr +  b"\x00"
    length_block = len_last_block.to_bytes(16, 'big') # Turns the length of the last block into a 16 bytes block
    c_aux = tpbc(tweak, key, length_block, iv) # c_aux: bytes
    
    c_m = b""
    for x,y in zip(block_plaintext[-1], c_aux):
        word = x ^ y
        c_m += word.to_bytes(1, 'big')
    ct += c_m # ct: bytes
    
    
    aux = b""
    for x,y in zip(auth, block_plaintext[-1],):
        word = x ^ y
        aux += word.to_bytes(1, 'big')
    auth = aux
    
    # Autenticação
    tweak = nounce + ctr + b"\x01"

    tag = tpbc(tweak, key, auth, iv)
    
    return {"ct": ct, "tag": tag, "nounce": nounce, "ctr": ctr_i, "pad": len_last_block, "iv": iv}

## Decifrar mensagem

In [6]:
# size_block = tau
def unpad(last_block, size_block, NBytes):
    
    last_block = last_block[:size_block]
    
    return last_block


def un_tpbc(tweak, key, block, iv):
    tweaked_key = tweak + key
    cipher = Cipher(algorithms.AES256(tweaked_key), modes.CBC(iv))
    decryptor = cipher.decryptor()
    plain_block = decryptor.update(block) + decryptor.finalize()
    return plain_block

def undo_tweakable_first_blocks(nounce, ctr, block_ciphertext, key, auth, iv):
    plaintext = b""    
    for elem in (block_ciphertext[:-1]):        
        tweak = nounce + ctr +  b"\x00"
        
        c_i = un_tpbc(tweak, key, elem, iv)
        plaintext += c_i
        
        len_ctr = len(ctr)
        ctr_int = int.from_bytes(ctr, 'big')
        ctr_int += 1
        ctr = ctr_int.to_bytes(len_ctr, 'big')
        
        aux = b""
        for x,y in zip(auth, c_i):
            word = x ^ y
            aux += word.to_bytes(1, 'big')
        
        auth = aux
        
    return ctr, auth, plaintext


def decrypt(msg, key):
    ct = msg['ct'] # bytes
    tag_rcv = msg['tag']
    nounce = msg['nounce']
    ctr = msg['ctr']
    len_last_block = msg['pad']
    iv = msg['iv']
    
    NBytes = 16
    
    block_ciphertext = [ct[i:i+NBytes] for i in range(0, len(ct), NBytes)]  # list of block of bytes. Block size 16 bytes
    
    # i= 0 ... m - 1
    auth = b"" 
    for _ in range (NBytes): # array of bytes with size 16 bytes
        auth += b"\x00"
    ctr, auth, plaintext = undo_tweakable_first_blocks(nounce, ctr, block_ciphertext, key, auth, iv)
        
    # i = m
    tweak = nounce + ctr + b"\x00"
    length_block = len_last_block.to_bytes(16, 'big')
    c_aux = tpbc(tweak, key, length_block, iv)
    
    c_m = b""
    for x,y in zip(block_ciphertext[-1], c_aux):
        word = x ^ y
        c_m += word.to_bytes(1, 'big')
    
    plaintext += c_m[:len_last_block] # ct: bytes
    
    aux = b""
    for x,y in zip(auth, c_m):
        word = x ^ y
        aux += word.to_bytes(1, 'big')
    auth = aux
    
    # Autenticação
    tweak = nounce + ctr + b"\x01"
    
    tag = tpbc(tweak, key, auth, iv)
    
    
    if tag != tag_rcv:
        print("Mensagem inválida")
    else:
        print("Mensagem válida")
       
    return plaintext

## Funções auxiliares para o canal de comunicação
Encontra-se 4 métodos auxiliares para:
- inicialização dos agentes;
- Envio de mensagens;
- Receção de mensagem.

Os métodos para envio e receção de mensagens
têm em consideração o código proporcionado
pela equipa docente, em consideração das 
estruturas _queues_.

O método para __inicialização__:
- Cria os dois pares de chaves pública-privada (para cifrar e assinar), para o _emitter_;
- Cria um par de chaves (para decifrar), para o _receiver_;
- Serializa as chaves públicas (para enviar ao _peer_);
- Retorna um tuplo com as chaves e o pacote a enviar ao peer.

...com o nome _init_comm()_emiiter_ e/ou _init_comm()_receiver_.

In [7]:
def init_comm_emitter():
    ## Gerar chaves (cifrar e autenticar)
    prv_cipher_key, pub_cipher_key = generateKeys()
    
    prv_sign_key, pub_sign_key = generateSignKeys()
    
    ## Dicionário com a chavess públicas (serializadas)
    msg = {'cipher_key': pub_cipher_key.public_bytes(encoding=serialization.Encoding.Raw,
                                       format=serialization.PublicFormat.Raw
                                    ), 
           'sign_key': pub_sign_key.public_bytes(encoding=serialization.Encoding.Raw,
                                       format=serialization.PublicFormat.Raw
                                    )
            }
    
    return prv_cipher_key, prv_sign_key, msg

def init_comm_receiver():
    ## Gerar chaves (cifrar e autenticar)
    prv_cipher_key, pub_cipher_key = generateKeys()
    
    ## Dicionário com a chavess públicas (serializadas)
    msg = {'cipher_key': pub_cipher_key.public_bytes(encoding=serialization.Encoding.Raw,
                                       format=serialization.PublicFormat.Raw
                                    )
          } 
    
    return prv_cipher_key, msg

async def send(queue, msg):
    
    await asyncio.sleep(random.random())
        
    # put the item in the queue
    await queue.put(msg)
    
    await asyncio.sleep(random.random())
    
async def receive(queue):
    item = await queue.get()

    await asyncio.sleep(random.random())
    aux = loads(item)

    return aux

## Emitter

In [8]:
## Emitter Code
async def emitter(plaintext, queue):
    
    ## Gerar as chaves (privada e publica) & partilhar com participante
    prv_cipher_key, prv_sign_key, msg = init_comm_emitter()
    
    ## Enviar a chaves públicas para o peer
    await send(queue, dumps(msg))
    print("[E] SENDING PUBLIC KEYS")
    
    ## Receber as chaves públicas do peer
    msg = await receive(queue)
    print("[E] RECEIVED PEER PUBLIC KEYS")
    
    pub_peer_cipher = msg['cipher_key']
    print("[E] Receiver pub_key_cipher: " +str(msg['cipher_key']))
    
    ## Criar as chaves partilhadas (cifrar/autenticar)
    cipher_shared = generateShared(prv_cipher_key, pub_peer_cipher)
    
    print("CIPHER_SHARED: "+str(cipher_shared))
    
    ## Cifrar a mensagem
    pkg = encrypt(plaintext, cipher_shared)
    print("[E] MESSAGE ENCRYPTED")
    
    ## Assinar e enviar a mensagem
    pkg_b = dumps(pkg)
    sig = signMsg(prv_sign_key, pkg_b)
    
    # Enviar
    msg_final = {'sig': sig, 'msg': dumps(pkg)}
    
    print("[E] SENDING MESSAGE")
    await send(queue, dumps(msg_final))
    
    print("[E] END")

## Receiver

In [9]:
## Receiver Code
async def receiver(queue):
    
    ## Gerar as chaves (privada e publica) & partilhar com participante
    prv_cipher_key, msg = init_comm_receiver()
    
    ## Receber as chaves publicas do peer
    pub_keys = await receive(queue)
    
    pub_peer_cipher = pub_keys['cipher_key']
    pub_peer_sign = pub_keys['sign_key']
    
    print("[R] Emitter pub_key_cipher: " +str(pub_peer_cipher))
    print("[R] Emitter pub_key_sign: " +str(pub_peer_sign))
    
    ## Gerar shared keys
    cipher_shared = generateShared(prv_cipher_key, pub_peer_cipher)

    ## Enviar as chaves públicas ao peer
    await send(queue, dumps(msg))
    print("[R] AWAIT CIPHER")
    ciphertext = await receive(queue)
    print("[R] CIPHER RECEIVED")
    
    ## Obter a chave pública (Assinatura)
    peer_sign_pubkey = ed448.Ed448PublicKey.from_public_bytes(pub_peer_sign)
    
    ## Validar a correçaõ da assinatura
    peer_sign_pubkey.verify(ciphertext['sig'], ciphertext['msg'])
       
    msg_dict = loads(ciphertext['msg'])
                      
    ## Decifrar essa mensagem       
    plain_text = decrypt(msg_dict, cipher_shared)
    
    ## Apresentar no terminal
    print("[R] Plaintext: " + plain_text.decode('utf-8'))
       

In [10]:
def test(msg):
    loop = asyncio.get_event_loop()
    queue = asyncio.Queue(10)
    asyncio.ensure_future(emitter(msg, queue), loop=loop)
    loop.run_until_complete(receiver(queue))
    
test("HELLO WORLD! THIS IS JUST A TEST OF A AEAD ALGORITHM.")

[E] SENDING PUBLIC KEYS
[R] Emitter pub_key_cipher: b'\xc6\x00L\xb4\xf7\x81t\xa5\xf1\xc99=\x82I6\\\xc7D4\xb1\x04v\xe8R%\x8a\x02\x9b\x03I+\\\xcas\tm\xa9\xae\x86j\xa3\xcd<\xc4\n+\xcf(w=\xe6ML\xb8\xc1\x9a'
[R] Emitter pub_key_sign: b'\xc4\x9d\x93\x95\xbdk\xa1\xff\x88\xc2\x14\xbd[\xf5\xe0\xd7\xccdvq?[\x06\xc0x\xe8X\x8c\x90\xe5\x9f\xe3y\xdc\xcd\xed\xb7\xe4\x81\xfal7yR\xa9C*\xfc\xbe\xee?_\xd1\xa53\x99\x80'
[R] AWAIT CIPHER
[E] RECEIVED PEER PUBLIC KEYS
[E] Receiver pub_key_cipher: b'nF\x01\xc4$\xf4\x8f\x83\xa3\xa9\x87\xa7Zky\xe8Yw\xaa\xf8\x1d\xe0\x80\xe7u\x16DA\xb8&\x8e\x06\xdb\x04)\xb9<t\x84\x89\xc8U\rV\xff\x84\x1b\xfa\xef\x1cM\x06\x08\xe6\x01\x00'
CIPHER_SHARED: b'\xa4\xb2\xe4\xe72=\x04\xc1\x82j\x01\xad)\xfc\xca\xb9'
[E] MESSAGE ENCRYPTED
[E] SENDING MESSAGE
[R] CIPHER RECEIVED
Mensagem válida
[R] Plaintext: HELLO WORLD! THIS IS JUST A TEST OF A AEAD ALGORITHM.
